In [1]:
import os, sys

src_path = os.path.abspath(os.path.join(os.path.dirname(__file__), "src"))
sys.path.insert(0, src_path)

from pandas import DataFrame
from analytics.metrics.metric import MetricCalculator
from analytics.metrics.time import TimeMetrics
from spec.enums import MainTableColumns as Cols

NameError: name '__file__' is not defined

In [ ]:

original_df = DataFrame()

time_metrics = TimeMetrics()

results = time_metrics.calculate(original_df.groupby([Cols.SubjectID, Cols.ProblemID]))